In [2]:
#######This code is for automatically calculating and exporting two of the features within 
##the machine learning model to an excel sheet ('friday short volume' and 'friday short volume-5 day average')

## It functions by taking in a list of stock tickers at the stock variable
##it then gets the short volume data from the fintel webpage 
##by digesting the html code and pulling out the relevant information
##for a respective stock
##Fintel has throttling on its website (will block your IP if you make too many requests
##during a given time frame) To deal with this i am making my requests through the 
##application Tor, and using it to change my IP address every request
##In order to use this you need to download the Tor application: https://www.torproject.org/download/
#and then download the torrequest python library and configure one of the files within the 
##torrequest package by adding the tor application executable file path to a line of 
#python code, process described in detail here: https://stackoverflow.com/questions/24801924/tor-with-python-stem-basic-tor-not-in-path
#(the second answer is the one that describes what i did)

#Inputs you need to make:
#1. specify export file path location
#2. specify your stock ticker list
#3. specify dates in your week
#4. specify friday
##This hasnt been completely trouble shot, it will throw a couple of different errors 
## that mostly deal with processing the html code. When that happens the code will export
##'NaN' for the feature export values associated with that ticker
## One error that has been successfully dealt with is 'days missing'. For whatever reason
##stocks sometimes won't have a day of short volume data, i deal with this by counting how many 
##times that this happens and exporting it to the sheet. 
#any stock with more than one day of short data missing and or friday short data missing gets
#doesn't get included in the machine learning model.  
import requests
import re 
import string
import time 
import numpy as np 
import pandas as pd 
import xlsxwriter
from torrequest import TorRequest
import requests
from torpy import TorClient
import os 
import torpy
from torpy.http.requests import TorRequests
from multiprocessing.pool import ThreadPool
from torpy.http.requests import tor_requests_session
from stem.control import Controller
import stem 


torprocess = stem.process.launch_tor(tor_cmd = '/Applications/Tor Browser.app/Contents/Resources/TorBrowser/Tor/tor')
cookies = dict(BCPermissionLevel='PERSONAL')
proxy = {'http':  'socks5://127.0.0.1:9050',
           'https': 'socks5://127.0.0.1:9050'}
   # return session
#################################specify export file location############################################
workbook=xlsxwriter.Workbook('/Users/jpjansen/Documents/Shorts_5_9_v1.xlsx')
##############################################################################################
worksheet = workbook.add_worksheet()
worksheet.write(0,0,'Stock Symbol')
worksheet.write(0,1,'Friday Short Ratio')
worksheet.write(0,2,'Friday-Week Average')
worksheet.write(0,3,'Week Average')
worksheet.write(0,4,'Days Missing')
######''#########stock list####################################################################
stock = ['XSPA', 'RKDA', 'ETH', 'RHE', 'CCXI', 'BBIG', 'OBLN', 'APOP', 'PRFX', 'YTEN', 'BGFV', 'VISL', 'CDEV', 'ANY', 'FTOC', 'CLOV', 'PRPO', 'BTX', 'BCRX', 'MGI', 'NKLA', 'LEDS', 'PLTR', 'MVIS', 'OCGN', 'CREG', 'SEAC', 'ADVM', 'SONN', 'CCIV']
url = "https://fintel.io/ss/us/"
allow = string.ascii_lowercase[1]+string.digits+"="+"."+"-"
friday_shorts = []
week_average = []
friday_subtractmean = [] 
counter = 0 

for k in range(len(stock)):
    loopURL = url+stock[k]
    with TorRequest(proxy_port=9050, ctrl_port=9051, password = 'password') as tr:
        res = tr.get(loopURL, proxies = proxy, headers={"User-Agent": "Mozilla/5.0"}, cookies=cookies)
        ipaddress= tr.get('http://ipecho.net/plain')
        print ("New Ip Address",ipaddress.text)
        tr.reset_identity()
        tr.close()
    print(stock[k])

    a = res.content
    b = a.decode('UTF-8')
    c = b.replace('"', '')
    c = c.replace("'", '')
    c = c.replace(",", '')
    c = c.replace(";",'')
    c = c.replace(">",'')
    c = c.replace("<",'')
    c = c.replace("/",'')
    c = c.replace("/",'')
    c = c.replace('','')
    c = c.replace("\r","")
    c = c.replace("\n","")
   
    try:
        d = c.split("Short Volume Ratio adivdiv class=col-xs-12 col-sm-12 col-md-12 col-lg-6table id=topic-table-body")
        e = d[1]
        f = e.split('tdtrtbodytabledivdivdivdiv')
    except IndexError:
        friday = 'NaN'
        week_average = 'NaN'
        friday_subtractmean = 'NaN'
        print(stock[k],'INDEX ERROR')
        
        worksheet.write(counter+1,0,stock[k])
        worksheet.write(counter+1,1,friday)
        worksheet.write(counter+1,2,friday_subtractmean)
        worksheet.write(counter+1,3,week_average)
        counter+=1
        time.sleep(1)
        continue 
  
    asdf = re.sub('[^%s]'% allow, '', f[0])
    testing = asdf.split('=')
    data = []
    for i in range(len(testing)):
        z = testing[i].split('b')
        for q in range(len(z)):
            if z[q]!='':
                if z[q]!='-':
                    data.append(z[q])
    #print(data)
###############################specify dates in week############################################################                
    previous_week_dates = ['2021-05-03','2021-05-04','2021-05-05','2021-05-06', '2021-05-07'] 
############################################################################################
    try:
        shortvolume = []
        totalvolume=[]
        shortratio = [] 
        for i in range(len(previous_week_dates)):
            for z in range(len(data)):
                if data[z]== previous_week_dates[i]:
                    #print('MATCH')
                    #print('SV', data[z+1])
                    #print('TV', data[z+2])
                    #print('SR', data[z+3])
                    shortvolume.append(data[z+1])
                    totalvolume.append(data[z+2])
                    shortratio.append(data[z+3])
                    break
        SVfloat = []
        TVfloat = []
        SRfloat = []
        for i in range(len(shortratio)):
            SVfloat.append(np.abs(float(shortvolume[i])))
            TVfloat.append(np.abs(float(totalvolume[i])))
            SRfloat.append(np.abs(float(shortratio[i])))
        #print(SVfloat,TVfloat,SRfloat)
        #dates = np.transpose(previous_week_dates)
        #final_data = np.column_stack((dates, SVfloat, TVfloat, SRfloat))
        #print(final_data)
        #df = pd.DataFrame(data=final_data[:,1:], index=[dates], columns=["short volume", "total volume", "short ratio"])
        try:
            dates = np.transpose(previous_week_dates)
            final_data = np.column_stack((dates, SVfloat, TVfloat, SRfloat))
            #print(final_data)
            df = pd.DataFrame(data=final_data[:,1:], index=[dates], columns=["short volume", "total volume", "short ratio"])
####################################specify friday date #############################################################
            friday = float(df.loc['2021-05-07','short ratio'].values[0])
            print('Friday = ', friday)
#################################################################################################
            friday_shorts.append(friday)
        except KeyError:
            friday = 'NaN'
            week_average = 'NaN'
            friday_subtractmean = 'NaN'
            print(stock[k],'INDEX ERROR')

            worksheet.write(counter+1,0,stock[k])
            worksheet.write(counter+1,1,friday)
            worksheet.write(counter+1,2,friday_subtractmean)
            worksheet.write(counter+1,3,week_average)
            counter+=1
            time.sleep(1)
            continue 
   
        short_ratios = [] 
        days_missing = 0 
        for i in range(len(previous_week_dates)):
            try:
                short_ratios.append(float(df.loc[previous_week_dates[i],'short ratio'].values[0]))
            except KeyError:
                days_missing+=1
        print('short ratios',short_ratios)
        print('Fridays',friday)
        week_average = np.sum(short_ratios)/len(short_ratios)
        print('week average', week_average)
        friday_subtractmean = friday-week_average
        print('friday minus mean',friday_subtractmean)
    
    except ValueError:
        print("Value Error", stock[k])
        week_average = 'NaN'
        friday_subtractmean = 'NaN'
    try:
        print(df)
    except NameError:
        friday = 'NaN'
        days_missing = 'NaN'
        print('Name Error')
    worksheet.write(counter+1,0,stock[k])
    worksheet.write(counter+1,1,friday)
    #print('friday_subtractmean', friday_subtractmean)
    worksheet.write(counter+1,2,friday_subtractmean)
    worksheet.write(counter+1,3,week_average)
    worksheet.write(counter+1,4,days_missing)
    worksheet.write(counter+1,5,ipaddress.text)

    counter+=1
    time.sleep(1)
workbook.close()


New Ip Address 185.220.101.12
XSPA
Friday =  10.89
short ratios [15.63, 14.17, 16.29, 16.56, 10.89]
Fridays 10.89
week average 14.708000000000002
friday minus mean -3.8180000000000014
           short volume total volume short ratio
2021-05-03     361968.0         2.32       15.63
2021-05-04     353242.0         2.49       14.17
2021-05-05     523771.0         3.21       16.29
2021-05-06     472877.0         2.86       16.56
2021-05-07     240846.0         2.21       10.89
New Ip Address 95.214.235.160
RKDA
Friday =  30.09
short ratios [25.34, 25.39, 32.7, 31.74, 30.09]
Fridays 30.09
week average 29.052
friday minus mean 1.0380000000000003
           short volume total volume short ratio
2021-05-03     270036.0         1.07       25.34
2021-05-04     202440.0          0.8       25.39
2021-05-05     337765.0         1.03        32.7
2021-05-06     212144.0         0.67       31.74
2021-05-07     183486.0         0.61       30.09
New Ip Address 185.220.101.218
ETH
Friday =  14.52
short r

New Ip Address 199.195.253.26
BCRX
Friday =  13.0
short ratios [11.31, 7.47, 16.0, 17.07, 13.0]
Fridays 13.0
week average 12.969999999999999
friday minus mean 0.030000000000001137
           short volume total volume short ratio
2021-05-03     273932.0         2.42       11.31
2021-05-04     199631.0         2.67        7.47
2021-05-05     409187.0         2.56        16.0
2021-05-06    3075210.0        18.01       17.07
2021-05-07     838360.0         6.45        13.0
New Ip Address 185.220.100.242
MGI
Friday =  27.89
short ratios [16.77, 16.89, 17.24, 24.92, 27.89]
Fridays 27.89
week average 20.741999999999997
friday minus mean 7.148000000000003
           short volume total volume short ratio
2021-05-03     254003.0         1.51       16.77
2021-05-04     245299.0         1.45       16.89
2021-05-05     254234.0         1.47       17.24
2021-05-06     567528.0         2.28       24.92
2021-05-07    6026230.0        21.61       27.89
New Ip Address 185.220.101.21
NKLA
Friday =  28.87

In [4]:
print(os.getcwd())

/Users/jpjansen
